In [512]:
!pip install PyPDF2
!pip install pymupdf Pillow pytesseract

In [513]:
from PyPDF2 import PdfReader
import fitz  # PyMuPDF
import os
import shutil

def get_pic(page_num):
    pdf_path = "QA.pdf"
    folder = "images"


    doc = fitz.open(pdf_path)
    
    image_list = doc.get_page_images(page_num)
    
    for img_index, img in enumerate(image_list):
        xref = img[0]
        pix = fitz.Pixmap(doc, xref)
        pix = fitz.Pixmap(fitz.csRGB, pix)
        pix.save(os.path.join(folder, "page{}_img{}.png".format(page_num-6, img_index)))

def read(file_path, start, end):
    
    result = []
    
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        num_pages = len(pdf_reader.pages)
        if end > num_pages:
            print("end number outside of total pages")
            return None


        folder = "images"

        if os.path.exists(folder):
            shutil.rmtree(folder)
    
        os.makedirs(folder)
        q = 0
        
        for page_num in range(start, end):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()

            if "Answers and Explanations" in text:
                print("bad")
                q = 0

            if "Multiple Choice Questions" in text:
                print("good")
                q = 1

            if q == 1:
                get_pic(page_num)
                q = 0
                
            result.append(text)

    return "\n".join(result)

In [514]:
textbook = "QA.pdf"
contents = read(textbook, 9, 191)
questions_d, answers_d = get_questions_and_answers(contents)
#answers_d = clean_sentences(answer)
#print(answers_d)

good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good


In [515]:
import re

def filter(text):
    def license_filter(text):
        pattern = r"© The Author\(s\), under exclusive license to Springer Nature Switzerland AG \d{4}\s+V\s*\.\s*K\s*\.\s*Kohli\s*et\s*al\.,\s*Comprehensive Multiple-Choice Questions in Pathology\s*,\s*https://doi\.org/10\.1007/[\d\w\-_.]+"
        return re.sub(pattern, "", text)

    text = text.replace("\xa0", " ")
    text = license_filter(text)
    return text


In [516]:
import numpy as np 

def get_questions_and_answers(text):
    questions = []
    answers = []

    current_Q = []
    current_A = []

    q = True
    a = False
    filtered_text = filter(text)

    for line in filtered_text.split('\n')[4:]:
        line = line.strip()
        if q:
            if "Answers and Explanations" in line:
                questions.append(current_Q)
            
                current_Q = []
                q = False
                a = True
                continue

            
            current_Q.append(line)

        if a:
            if "Bibliography" in line:
                answers.append(current_A)
                current_A = []
                q = False
                a = False
                continue
                
            current_A.append(line)

        if q * a == 0:
            if "Multiple Choice Questions" in line:
                q = True
                a = False
            
    return questions, answers

In [517]:
def restructure_questions(questions):
    result = []
    
    def split_q(text):
        pattern = r"(?=\b\d{1,2}\.\s|A\.\s*|B\.\s*|C\.\s*|D\.\s*|E\.\s*)"
        split_text = re.split(pattern, text)
        return [part.strip() for part in split_text if part.strip() and len(part) > 1]

    y = 1
    
    for question_block in questions:
        text = " ".join(question_block)
        lists = split_q(text)
        current = []
        x = 1
        
        for i in lists:
            if re.match(r'^\d{1,2}\.\s', i):
                if current:
                    result.append([f"Set_{y}: q{x}"] + current)
                    x += 1
                current = [i[3:]]
            else: 
                current.append(i)
        
        if current:
            result.append([f"Set_{y}: q{x}"] + current)
        y += 1
    
    return result

In [518]:
def restructure_answers(answers):
    result = []
    def split_a(text):
        pattern = r'(?<!\d)\b\d+\.\s+(?=[A])'
        split_text = re.split(pattern, text)
        return np.array([part.strip() for part in split_text if part.strip()])
    y = 1
    for answer in answers:
        x = 1
        text = "\n".join(answer)
        lists = split_a(text)
            

        
        for line in lists:
           # line = re.sub(r'^Answer: ', '', line)
            current = [f"Set_{y}: q{x}"] + [line[:line.find("\n")], line[line.find("\n")+1:].replace("\n", " ")]
            result.append(current)
            x += 1
        y += 1
    return result
        

In [519]:
def remove_answers_and_explanations(text):
    pattern1 = r'Answers and Explanations.*?((?<!\.)\s|$)'
    cleaned_text = re.sub(pattern1, '', text, flags=re.DOTALL).strip()
    #pattern = r"(?<![A-E]\.)(?<=\.\s*)[^\s.]+(\s+[^\s.]+)*\s*$"
    #cleaned_text = re.sub(pattern, '.', text).strip()
    return cleaned_text

def clean_sentences(double_list):
    return [[remove_answers_and_explanations(sentence) for sentence in sublist] for sublist in double_list]



def ending(text):
    pattern = r"\.\s*[^\s.]+(\s+[^\s.]+)*\s*$"
    modified_text = re.sub(pattern, '.', text).strip()
    return modified_text

def explanation(answers):
    for answer in answers:
        if len(answer) >= 3:
            answer[2] = ending(answer[2])
    return answers



In [520]:
textbook = "QA.pdf"
contents = read(textbook, 9, 191)
questions_d, answers_d = get_questions_and_answers(contents)
#answers_d = clean_sentences(answer)
#print(answers_d)

good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good


In [521]:
questions = restructure_questions(questions_d)

In [522]:
answers = restructure_answers(answers_d)
answers = clean_sentences(answers)
answers = explanation(answers)
#print(answers)

In [523]:
import pandas as pd

q_table = pd.DataFrame(data=questions)
q_table.columns = ["id", "Question", "A", "B", "C", "D", "E", "F"]

a_table = pd.DataFrame(data=answers)
a_table.columns = ["id", "Answer", "explaination"]

In [524]:
a_table = a_table[a_table["Answer"].str[:6] == "Answer"].reset_index()

In [525]:
q_table.dropna(how='all')
q_table.to_csv('questions.csv', index=False)

In [526]:
a_table.dropna(how='all')
a_table.to_csv('answers.csv', index=False)

In [527]:
a_table_m = pd.read_csv('answers.csv')
a_table_m['Answer'] = a_table_m['Answer'].str.replace(r'^Answer:\s*', '', regex=True)
a_table_m.to_csv('answers.csv', index=False)

In [528]:
new_row = {
    'id': 'Set_22: q2', 
    'Answer': 'C. Meningioma', 
    'explaination': 'Meningiomas are relatively common neoplasms derived from meningothelial cells of the arachnoid. The World Health organization (WHO) classified meningiomas in three groups'
}

a_table_m = pd.concat([a_table_m.iloc[:572],row, a_table_m.iloc[572:]]).reset_index(drop=True)
a_table_m.to_csv('answers.csv', index=False) ##这里跑一次就行


question = [[1,25],[4,5],[4,21],[5,13],[7,5],[7,6],[7,21],[8,14],[8,20],[8,24],[8,46],[9,6],[9,16],[10,9],[10,10],[10,14],[10,14],[12,6],[12,11],[12,12],[12,32],[14,1],[14,13],[14,20],[14,28],[16,21],[16,25],[17,11],[18,7],[18,10],[19,11],[20,3],[20,4],[20,10],[20,11],[20,12],[20,12],[20,13],[20,17],[20,18],[20,19],[20,20],[20,21],[20,23],[20,24],[20,26],[20,27],[20,28],[22,4],[22,27],[22,28],[23,3],[23,5],[23,11],[23,12]]
a_table_m['image'] = ' '

for set in question:
    set_id = 'Set_{}: q{}'.format(set[0], set[1])
    if not a_table_m[a_table_m['id'] == set_id].empty:
        row_index = a_table_m[a_table_m['id'] == set_id].index[0]
        a_table_m.at[row_index, 'image'] = '/images/Set_{}_q{}.png'.format(set[0], set[1])

a_table_m.to_csv('answers.csv', index=False)


In [529]:
q_table_f = pd.read_csv('questions_f.csv')
a_table_f = pd.read_csv('answers.csv')

In [530]:
table = pd.concat([q_table_f.drop(["id"], axis = 1), a_table_f.drop(["id"], axis = 1)], axis=1)
table.to_csv('final.csv', index=False)

In [531]:
a_table_f.drop(columns=['index'])
table_json = pd.concat([q_table_f.drop(["id"], axis = 1), a_table_f.drop(["id"], axis = 1)], axis=1)
table_json.to_csv('final.csv', index=False)

image test below

In [533]:
pip install pymupdf

Note: you may need to restart the kernel to use updated packages.


In [534]:
"""
import fitz  # PyMuPDF
import os
import shutil

# Path to the PDF file
pdf_path = "QA.pdf"
# Path to the folder where images will be saved
folder = "images"

if os.path.exists(folder):
    shutil.rmtree(folder)
    
os.makedirs(folder)

doc = fitz.open(pdf_path)

for i in range(10,190):
    # Get the list of images on the page
    image_list = doc.get_page_images(i)
    
    for img_index, img in enumerate(image_list):
        xref = img[0]
        pix = fitz.Pixmap(doc, xref)
        pix = fitz.Pixmap(fitz.csRGB, pix)
        pix.save(os.path.join(folder, "page{}_img{}.png".format(i-8, img_index)))

print("Image extraction completed.")
"""





'\nimport fitz  # PyMuPDF\nimport os\nimport shutil\n\n# Path to the PDF file\npdf_path = "QA.pdf"\n# Path to the folder where images will be saved\nfolder = "images"\n\nif os.path.exists(folder):\n    shutil.rmtree(folder)\n    \nos.makedirs(folder)\n\ndoc = fitz.open(pdf_path)\n\nfor i in range(10,190):\n    # Get the list of images on the page\n    image_list = doc.get_page_images(i)\n    \n    for img_index, img in enumerate(image_list):\n        xref = img[0]\n        pix = fitz.Pixmap(doc, xref)\n        pix = fitz.Pixmap(fitz.csRGB, pix)\n        pix.save(os.path.join(folder, "page{}_img{}.png".format(i-8, img_index)))\n\nprint("Image extraction completed.")\n'

In [535]:
last_num = -1
img_index = 0
counter = 0

def get_pic(page_num):
    global last_num, img_index,counter
    question = [[1,25],[4,5],[4,21],[5,13],[7,5],[7,6],[7,21],[8,14],[8,20],[8,24],[8,46],[9,6],[9,16],[10,9],[10,10],[10,14],[10,14],[12,6],[12,11],[12,12],[12,32],[14,1],[14,13],[14,20],[14,28],[16,21],[16,25],[17,11],[18,7],[18,10],[19,11],[20,3],[20,4],[20,10],[20,11],[20,12],[20,12],[20,13],[20,17],[20,18],[20,19],[20,20],[20,21],[20,23],[20,24],[20,26],[20,27],[20,28],[22,4],[22,27],[22,28],[23,3],[23,5],[23,11],[23,12]]

    pdf_path = "QA.pdf"
    folder = "images"

    if last_num == page_num:
        img_index += 1
    else:
        img_index = 0

    special_page= (49, 94, 112, 142, 155, 172, 184)
    if page_num in special_page:
        img_index = 1
        if last_num == page_num:
            img_index += 1

    doc = fitz.open(pdf_path)
    
    image_list = doc.get_page_images(page_num)
    
    img = image_list[img_index]
    
    xref = img[0]
    pix = fitz.Pixmap(doc, xref)
    pix = fitz.Pixmap(fitz.csRGB, pix)
    set = question[counter]
    counter += 1
    pix.save(os.path.join(folder, "Set_{}_q{}.png".format(set[0], set[1])))

    last_num = page_num
    
    
folder = "images"

if os.path.exists(folder):
    shutil.rmtree(folder)
    
os.makedirs(folder)

for i in page_num:
    get_pic(i)

In [536]:
page_num = [11,29,31,36,49,50,51,57,58,58,60,71,73,79,79,79,80,94,95,95,97,112,113,114,115,129,130,136,142,143,149,155,155,156,156,157,157,157,158,158,158,158,159,159,159,160,160,160,172,174,175,184,184,185,185]
question = [[1,25],[4,5],[4,21],[5,13],[7,5],[7,6],[7,21],[8,14],[8,20],[8,24],[8,46],[9,6],[9,16],[10,9],[10,10],[10,14],[10,14],[12,6],[12,11],[12,12],[12,32],[14,1],[14,13],[14,20],[14,28],[16,21],[16,25],[17,11],[18,7],[18,10],[19,11],[20,3],[20,4],[20,10],[20,11],[20,12],[20,12],[20,13],[20,17],[20,18],[20,19],[20,20],[20,21],[20,23],[20,24],[20,26],[20,27],[20,28],[22,4],[22,27],[22,28],[23,3],[23,5],[23,11],[23,12]]